In [1]:
%load_ext autoreload
%autoreload 2
from mail_agent.app import init_app, MailAgentApp
from mail_agent.models import User
from mail_agent.gmail import Gmail
from mail_agent.cli import get_user_by_email


In [15]:
app = init_app()

# Check if user needs re-authentication
with app.app_context():
    user = get_user_by_email(app, '')
    
    print(f"Current user status:")
    print(f"- Has access token: {bool(user.access_token)}")
    print(f"- Has refresh token: {bool(user.refresh_token)}")
    
    if user.access_token and not user.refresh_token:
        print("\n⚠️  ISSUE: User has access token but no refresh token!")
        print("This means the token will expire and cannot be refreshed.")
        print("\nSOLUTION: User needs to re-authenticate to get a refresh token.")
        print("Visit: http://localhost:5000/reauth")
    elif user.refresh_token:
        print("\n✅ User has both access and refresh tokens - should work properly!")
    else:
        print("\n❌ User has no tokens - needs to authenticate first.")
        print("Visit: http://localhost:5000/login")

Current user status:
- Has access token: True
- Has refresh token: False

⚠️  ISSUE: User has access token but no refresh token!
This means the token will expire and cannot be refreshed.

SOLUTION: User needs to re-authenticate to get a refresh token.
Visit: http://localhost:5000/reauth


In [18]:

with app.app_context():
    user = get_user_by_email(app, '')
    gmail = Gmail(app, user.id)
    envs = gmail.fetch_envelopes(max_results=1000)
        
len(envs)

1000

<GmailMessage id=196ffd319f98937d subject='Let's Encrypt certificate expiration notice for do...'>

In [16]:




with app.app_context():
    
    print(user)
    
    gmail = Gmail(app, user.id)
    emails = gmail.fetch_all(max_results=1000)
        
len(emails)

<User eric.busboom@jointheleague.org>


1000

In [20]:
import re
import email
import pandas as pd

addrs = []
for e in envs:

    match = re.search(r'<([^>]+)>', e.sender)
    if  match:
        addr = match.group(1)
    else:
        addr = e.sender

    domain = addr.split('@')[-1] if '@' in addr else ''

    addrs.append({'address':addr,
                  'domain':domain,
                  })
    
df = pd.DataFrame(addrs)
df

,address,domain
0,expiry@letsencrypt.org,letsencrypt.org
1,brittnesheim@gmail.com,gmail.com
2,eric.busboom@jointheleague.org,jointheleague.org
3,ads-account-noreply@google.com,google.com
4,jed.stumpf@jointheleague.org,jointheleague.org
...,...,...
995,account-services@inform.bill.com,inform.bill.com
996,mark.lantsberger@sdcoe.net,sdcoe.net
997,eric.busboom@jointheleague.org,jointheleague.org
998,mark.lantsberger@sdcoe.net,sdcoe.net


In [22]:
df.domain.value_counts().head(40)

domain
jointheleague.org                 372
gmail.com                         126
meetup.com                         72
hightechhigh.org                   17
letsencrypt.org                    16
hotmail.com                        16
rnt.myworkplace.co                 16
zoom.us                            14
googlegroups.com                   14
connectedcommunity.org             12
promo.newegg.com                   10
inform.bill.com                     8
email.meetup.com                    8
amazon.com                          8
dochub.com                          8
asynccomm.zoom.us                   8
coder.build                         8
medium.com                          7
schettlermacy.com                   7
discord.com                         7
grantgopher.com                     7
gusto.com                           6
business.amazon.com                 6
glassdoor.com                       6
peachjar.com                        6
udacity.com                         5
grant

In [11]:
import time

# Test performance of optimized fetch_envelopes
with app.app_context():
    gmail = Gmail(app, user.id)
    
    # Test fetch_envelopes performance with smaller batches
    print("Testing optimized fetch_envelopes (smaller batches)...")
    start_time = time.time()
    envelopes = gmail.fetch_envelopes(max_results=100)
    envelope_time = time.time() - start_time
    
    print(f"fetch_envelopes: {len(envelopes)} emails in {envelope_time:.2f} seconds")
    
    # Show some sample data
    print("\nSample envelope data:")
    for i, env in enumerate(envelopes[:3]):
        print(f"{i+1}. From: {env.sender[:50]}")
        print(f"   Subject: {env.subject[:50]}")
        print(f"   Date: {env.date_str}")
        print()

Testing optimized fetch_envelopes (smaller batches)...


Failed to fetch message envelope 196ffd319f98937d: <HttpError 429 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/196ffd319f98937d?format=metadata&metadataHeaders=From&metadataHeaders=To&metadataHeaders=Subject&metadataHeaders=Date&metadataHeaders=Message-ID&alt=json returned "Too many concurrent requests for user.". Details: "[{'message': 'Too many concurrent requests for user.', 'domain': 'global', 'reason': 'rateLimitExceeded'}]">
Failed to fetch message envelope 196ffa0190c86db3: <HttpError 429 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/196ffa0190c86db3?format=metadata&metadataHeaders=From&metadataHeaders=To&metadataHeaders=Subject&metadataHeaders=Date&metadataHeaders=Message-ID&alt=json returned "Too many concurrent requests for user.". Details: "[{'message': 'Too many concurrent requests for user.', 'domain': 'global', 'reason': 'rateLimitExceeded'}]">
Failed to fetch message envelope 196ff5117e31fe4c: <HttpError 429 when reque

fetch_envelopes: 100 emails in 1.03 seconds

Sample envelope data:
1. From: Eric Busboom <eric.busboom@jointheleague.org>
   Subject: Re: Brix S Kepler's Law Summer Python Project
   Date: Fri, 23 May 2025 16:58:13 -0700

2. From: Google Ads <ads-account-noreply@google.com>
   Subject: Your quarterly Security Suggestions, (Wintriss Tec
   Date: Fri, 23 May 2025 15:55:38 -0700

3. From: Jed Stumpf <jed.stumpf@jointheleague.org>
   Subject: Re: Teaching Programming
   Date: Fri, 23 May 2025 15:53:55 -0700



In [8]:
# Debug authentication issues
import json
from datetime import datetime

with app.app_context():
    user = get_user_by_email(app, '')
    print(f"User: {user}")
    print(f"User ID: {user.id if user else 'None'}")
    print(f"Has access token: {bool(user.access_token if user else False)}")
    print(f"Has refresh token: {bool(user.refresh_token if user else False)}")
    
    if user and user.access_token:
        print(f"Access token length: {len(user.access_token)}")
        print(f"Access token starts with: {user.access_token[:20]}...")
        
        # Try to get Gmail service to see if credentials work
        try:
            gmail = Gmail(app, user.id)
            service = gmail._get_service()
            print("✅ Successfully created Gmail service")
            
            # Try a simple API call
            profile = service.users().getProfile(userId='me').execute()
            print(f"✅ API call successful - Email: {profile.get('emailAddress')}")
            
        except Exception as e:
            print(f"❌ Error creating service or making API call: {e}")
            print(f"Error type: {type(e).__name__}")
    else:
        print("❌ No user found or missing tokens")

User: <User eric.busboom@jointheleague.org>
User ID: 1
Has access token: True
Has refresh token: False
Access token length: 222
Access token starts with: ya29.a0AW4XtxiTfC1Rd...
✅ Successfully created Gmail service
✅ API call successful - Email: eric.busboom@jointheleague.org


In [9]:
# Test batch request with authentication debugging
import time

with app.app_context():
    gmail = Gmail(app, user.id)
    
    print("Testing fetch_envelopes with detailed error handling...")
    start_time = time.time()
    
    try:
        # Try with a smaller number first
        envelopes = gmail.fetch_envelopes(max_results=10)
        elapsed = time.time() - start_time
        
        print(f"✅ Success: {len(envelopes)} envelopes fetched in {elapsed:.2f} seconds")
        
        if envelopes:
            print("\nSample envelope:")
            env = envelopes[0]
            print(f"From: {env.sender}")
            print(f"Subject: {env.subject}")
            print(f"Date: {env.date_str}")
        
    except Exception as e:
        elapsed = time.time() - start_time
        print(f"❌ Error after {elapsed:.2f} seconds: {e}")
        print(f"Error type: {type(e).__name__}")
        
        # Print more details if it's a Google API error
        if hasattr(e, 'resp'):
            print(f"HTTP Status: {e.resp.status}")
            print(f"Response content: {e.content.decode('utf-8') if hasattr(e, 'content') else 'No content'}")
        
        import traceback
        print("\nFull traceback:")
        print(traceback.format_exc())

Testing fetch_envelopes with detailed error handling...
✅ Success: 10 envelopes fetched in 0.20 seconds

Sample envelope:
From: Britt Nesheim <brittnesheim@gmail.com>
Subject: Re: Brix S Kepler's Law Summer Python Project
Date: Fri, 23 May 2025 17:09:51 -0700


In [10]:
# Test with different batch sizes to find the sweet spot
with app.app_context():
    gmail = Gmail(app, user.id)
    
    for test_size in [50, 100, 200]:
        print(f"\nTesting {test_size} emails...")
        start_time = time.time()
        
        try:
            envelopes = gmail.fetch_envelopes(max_results=test_size)
            elapsed = time.time() - start_time
            rate = len(envelopes) / elapsed if elapsed > 0 else 0
            
            print(f"✅ Success: {len(envelopes)} envelopes in {elapsed:.2f}s ({rate:.1f} emails/sec)")
            
        except Exception as e:
            elapsed = time.time() - start_time
            print(f"❌ Failed after {elapsed:.2f}s: {e}")
            break

Failed to find email envelopes for user 1: The credentials do not contain the necessary fields need to refresh the access token. You must specify refresh_token, token_uri, client_id, and client_secret.



Testing 50 emails...
✅ Success: 0 envelopes in 0.15s (0.0 emails/sec)

Testing 100 emails...


Failed to find email envelopes for user 1: The credentials do not contain the necessary fields need to refresh the access token. You must specify refresh_token, token_uri, client_id, and client_secret.
Failed to find email envelopes for user 1: The credentials do not contain the necessary fields need to refresh the access token. You must specify refresh_token, token_uri, client_id, and client_secret.
Failed to find email envelopes for user 1: The credentials do not contain the necessary fields need to refresh the access token. You must specify refresh_token, token_uri, client_id, and client_secret.


✅ Success: 0 envelopes in 0.07s (0.0 emails/sec)

Testing 200 emails...
✅ Success: 0 envelopes in 0.06s (0.0 emails/sec)


Current user status:
- Has access token: True
- Has refresh token: False

⚠️  ISSUE: User has access token but no refresh token!
This means the token will expire and cannot be refreshed.

SOLUTION: User needs to re-authenticate to get a refresh token.
Visit: http://localhost:5000/reauth
